In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.optimizers import Adam

import pickle
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import csv
from PIL import Image


Using TensorFlow backend.


In [2]:
(train_set, train_label_raw), (X_test, y_test) = mnist.load_data()
train_label = to_categorical(train_label_raw)
train_data = np.asarray(train_set).reshape(60000, 28, 28, 1)

In [3]:
gen_images_raw = pd.read_csv('generated_images_47600_0.923000.csv', header=None)
X_fake_raw = np.asarray(gen_images_raw).reshape(100000, 28, 28, 1)

gen_labels_raw = pd.read_csv('generated_labels_47600_0.923000.csv', header=None)
y_fake_raw = to_categorical(gen_labels_raw)
X_fake, y_fake = shuffle(X_fake_raw, y_fake_raw)

In [4]:
class classifier():
    def __init__(self):
        print("new classif")

    def train(self, totTrain=100_000, epochs=10, ratio=(1,1)): #Real to Fake
#         model = Sequential()
#         # model.add(Conv2D(5, kernel_size=3, activation='relu', input_shape=(28,28,1)))
#         model.add(Conv2D(64, kernel_size=3,  input_shape=(28,28,1)))
#         model.add(MaxPooling2D(pool_size=(4, 4)))
#         model.add(Conv2D(32, kernel_size=3, activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2, 2)))
#         model.add(Flatten())
#         model.add(Dense(10, activation='softmax'))
#         model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])    
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=(28,28,1)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy']) 

#         model.load_weights('initial_weights_2.h5')
        sum = ratio[0] + ratio[1]
        realTrain = ( totTrain // sum ) * ratio[0] 
        fakeTrain = ( totTrain // sum ) * ratio[1] 

        X_train_raw, X_val, y_train_raw, y_val = train_test_split(train_data, train_label, test_size=0.33, random_state=43)

        X_train_cat = np.concatenate((X_train_raw[:realTrain], X_fake[:fakeTrain]))
        y_train_cat = np.concatenate((y_train_raw[:realTrain], y_fake[:fakeTrain]))
        X_train, y_train = shuffle(X_train_cat, y_train_cat)
        print(X_train.shape)
        print(X_train_raw[:realTrain].shape)
        print(X_fake[:fakeTrain].shape)

        # model.summary()

        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
        os.makedirs('classif_history/', exist_ok=True)

        root = 'classif_history/'
        with open(root + 'classif_'+ str(totTrain) + '_' + str(ratio[0]) + '_' + str(ratio[1]) + '.csv', 'w') as f:
            for key in model.history.history.keys():
                f.write("%s,%s\n"%(key,model.history.history[key]))

        self.X_test_reshaped = X_test.reshape(10000, 28, 28, 1)
        loss, acc = model.evaluate(self.X_test_reshaped, to_categorical(y_test))
   
        print(acc)
        with open(root + 'classif_acc.csv', 'a') as b:
            b.write(str(totTrain) + ',' + str(ratio[0]) + ',' + str(ratio[1]) + ',' + "%f\n"%(acc))
            

In [ ]:
ratio_array = [ (0,1), (1,0), (1,1), (2,1), (1,2), (3,1), (1,3), (5,1), (1,5), (10,1), (1,10)]
train_array = [ 10000 ]



cl = classifier()

for num in train_array:
    for ratio in ratio_array:
#         if (ratio != ratio_array[0]) & (num != train_array[0]):
#             del cl
        
        cl.train(totTrain=num, epochs=10, ratio=ratio)
#         cl.evaluate()

new classif
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(10000, 28, 28, 1)
(0, 28, 28, 1)
(10000, 28, 28, 1)
Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 19800 samples
Epoch 1/10
10000/10000 [==============================] - 7s 734us/step - loss: 7.9205 - acc: 0.4489 - val_loss: 0.3863 - val_acc: 0.9047
Epoch 2/10
10000/10000 [==============================] - 4s 410us/step - loss: 0.2181 - acc: 0.9399 - val_loss: 0.4616 - val_acc: 0.9168
Epoch 3/10
10000/10000 [==============================] - 4s 412us/step - loss: 0.1250 - acc: 0.9657 - val_loss: 0.4066 - val_acc: 0.9362
Epoch 4/10
10000/10000 [==============================] - 4s 413us/step - loss: 0.1000 - acc: 0.9761 - val_loss: 0.4471 - val_acc: 0.9348
Epoch 5/10
10000/10000 [==============================] - 4s 415us/step - loss: 0.0852 - acc: 0.98

9999/9999 [==============================] - 4s 422us/step - loss: 0.1255 - acc: 0.9716 - val_loss: 0.1998 - val_acc: 0.9608
Epoch 9/10
9999/9999 [==============================] - 4s 425us/step - loss: 0.1146 - acc: 0.9726 - val_loss: 0.2163 - val_acc: 0.9595
Epoch 10/10
10000/10000 [==============================] - 1s 82us/step
0.9657
(10000, 28, 28, 1)
(7500, 28, 28, 1)
(2500, 28, 28, 1)
Train on 10000 samples, validate on 19800 samples
Epoch 1/10
10000/10000 [==============================] - 5s 479us/step - loss: 5.9844 - acc: 0.5629 - val_loss: 0.3297 - val_acc: 0.9155
Epoch 2/10
10000/10000 [==============================] - 4s 425us/step - loss: 0.3026 - acc: 0.9151 - val_loss: 0.2458 - val_acc: 0.9487
Epoch 3/10
10000/10000 [==============================] - 4s 424us/step - loss: 0.2207 - acc: 0.9419 - val_loss: 0.1413 - val_acc: 0.9617
Epoch 4/10
10000/10000 [==============================] - 4s 423us/step - loss: 0.1832 - acc: 0.9524 - val_loss: 0.1507 - val_acc: 0.9594
Epo

Train on 9999 samples, validate on 19800 samples
Epoch 1/10
9999/9999 [==============================] - 5s 495us/step - loss: 11.5044 - acc: 0.2731 - val_loss: 3.3904 - val_acc: 0.7268
Epoch 2/10
9999/9999 [==============================] - 4s 425us/step - loss: 0.5218 - acc: 0.8836 - val_loss: 0.2985 - val_acc: 0.9325
Epoch 3/10
9999/9999 [==============================] - 4s 426us/step - loss: 0.1655 - acc: 0.9567 - val_loss: 0.2920 - val_acc: 0.9363
Epoch 4/10
9999/9999 [==============================] - 4s 425us/step - loss: 0.1103 - acc: 0.9715 - val_loss: 0.3069 - val_acc: 0.9470
Epoch 5/10
9999/9999 [==============================] - 4s 425us/step - loss: 0.1072 - acc: 0.9741 - val_loss: 0.3184 - val_acc: 0.9474
Epoch 6/10
9999/9999 [==============================] - 4s 426us/step - loss: 0.0829 - acc: 0.9821 - val_loss: 0.4187 - val_acc: 0.9415
Epoch 7/10
9999/9999 [==============================] - 4s 426us/step - loss: 0.0783 - acc: 0.9831 - val_loss: 0.3394 - val_acc: 0.948

In [ ]:
ratio_array = [ (1,2) ]
train_array = [ 150_000 ]



for num in train_array:
    for ratio in ratio_array:
        del cl
#         if (ratio != ratio_array[0]) & (num != train_array[0]):
#             del cl
        cl = classifier()
        cl.train(totTrain=num, epochs=25, ratio=ratio)


new classif
(140200, 28, 28, 1)
(40200, 28, 28, 1)
(100000, 28, 28, 1)
Train on 140200 samples, validate on 19800 samples
Epoch 1/25
140200/140200 [==============================] - 40s 284us/step - loss: 1.3224 - acc: 0.8843 - val_loss: 0.1619 - val_acc: 0.9613
Epoch 2/25
140200/140200 [==============================] - 39s 278us/step - loss: 0.1271 - acc: 0.9680 - val_loss: 0.1694 - val_acc: 0.9620
Epoch 3/25
124992/140200 [=========================>....] - ETA: 4s - loss: 0.1447 - acc: 0.9656